In [1]:
import csv
import pandas as pd
import re
import sqlite3
from sqlite3 import Error
import ast 
import math
import os
import numpy as np

### Load Clean Data

In [2]:
receipts_df = pd.read_csv("cleaned_receipts.csv")
users_df = pd.read_csv("cleaned_users.csv")
brands_df = pd.read_csv("cleaned_brands.csv")

### Data Preparation

#### 1): find unique combination of category and category code, this is used for category relational table

In [3]:
brands_category_df = brands_df[brands_df['category'].notna() | brands_df['categoryCode'].notna()]
brands_category_list = set((row['category'], row['categoryCode']) for _, row in brands_category_df.iterrows())
#brands_category_list

#### 2): create a function to help find id in its original table

In [4]:
def get_id(table, value1, column_name1, value2=None, column_name2 = None):
    if table == 'category':
        column_name1 = 'category_name'
        column_name2 = 'category_code'
        cursor.execute(f"SELECT {table}_id FROM {table} WHERE {column_name1, column_name2} = (?,?)", (value1,value2))
    cursor.execute(f"SELECT {table}_id FROM {table} WHERE {column_name1} = ?", (value1,))
    result = cursor.fetchone()
    return result[0] if result else None

#### 3): create a function to help find brand code based on description in item

In [5]:
#used to map description to brand_code if brand_code not found from brand table
def get_brandcode(description):
    normalized_description = ' '.join(description.lower().split())
    words = normalized_description.split()
    for i in range(1, len(words) + 1):
        search_phrase = ' '.join(words[:i])
        query = f"""
        SELECT brand_code
        FROM brand
        WHERE LOWER(name) = LOWER(?)
        """
        cursor.execute(query, (search_phrase,))
        result = cursor.fetchone()
        if result:
            return result[0]
    return None

#### 4): create a function to help find brand id based on brand code

In [6]:
#used to map description to brand_code if brand_code not found from brand table
def get_brand_id(brand_code):
    query=f"""
    SELECT brand_id
    FROM brand
    WHERE brand_code = (?)
    """
    cursor.execute(query, (brand_code,))
    result = cursor.fetchone()
    
    if result:
        return result[0]
    return None

### Relational Database Schema Creation

In [9]:
conn = sqlite3.connect("receipts.db")
cursor = conn.cursor()
cursor.execute("PRAGMA foreign_keys = ON;")
#receipts table
cursor.execute("""drop table if exists bonus_reason """)
cursor.execute("""drop table if exists item """)
cursor.execute("""
CREATE TABLE IF NOT EXISTS bonus_reason (
    bonus_reason_id INTEGER PRIMARY KEY AUTOINCREMENT,
    bonus_reason_name TEXT NOT NULL UNIQUE
);
""")

cursor.execute("""drop table if exists receipt_status """)
cursor.execute("""
CREATE TABLE IF NOT EXISTS receipt_status (
    receipt_status_id INTEGER PRIMARY KEY AUTOINCREMENT,
    receipt_status_name TEXT NOT NULL UNIQUE
);
""")

#user table
cursor.execute("""drop table if exists state """)
cursor.execute("""
CREATE TABLE IF NOT EXISTS state (
    state_id INTEGER PRIMARY KEY AUTOINCREMENT,
    state_name TEXT NOT NULL UNIQUE
);
""")

cursor.execute("""drop table if exists role """)
cursor.execute("""
CREATE TABLE IF NOT EXISTS role (
    role_id INTEGER PRIMARY KEY AUTOINCREMENT,
    role_name TEXT NOT NULL UNIQUE
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS sign_up_source (
    sign_up_source_id INTEGER PRIMARY KEY AUTOINCREMENT,
    sign_up_source_name TEXT NOT NULL UNIQUE
);
""")

cursor.execute("""drop table if exists user """)
cursor.execute("""
CREATE TABLE IF NOT EXISTS user (
    user_id TEXT PRIMARY KEY,
    active INTEGER,
    created_date DATE NOT NULL,
    last_login_date DATE,
    role_id INTEGER,
    sign_up_source_id INTEGER,
    state_id INTEGER,
    FOREIGN KEY (role_id) REFERENCES role(role_id) ON DELETE SET NULL ON UPDATE CASCADE,
    FOREIGN KEY (sign_up_source_id) REFERENCES sign_up_source(sign_up_source_id) ON DELETE SET NULL ON UPDATE CASCADE,
    FOREIGN KEY (state_id) REFERENCES state(state_id) ON DELETE SET NULL ON UPDATE CASCADE
);
""")

#brand table
cursor.execute("""drop table if exists category """)
cursor.execute("""
CREATE TABLE IF NOT EXISTS category (
    category_id INTEGER PRIMARY KEY AUTOINCREMENT,
    category_name TEXT NOT NULL UNIQUE,
    category_code TEXT,
    UNIQUE (category_name, category_code)
);
""")

cursor.execute("""drop table if exists brand """)
cursor.execute("""
CREATE TABLE IF NOT EXISTS brand (
    brand_id TEXT PRIMARY KEY,
    barcode TEXT,
    cpg TEXT NOT NULL,
    name TEXT,
    top_brand INTEGER,
    brand_code TEXT,
    category_id INTEGER,
    FOREIGN KEY (category_id) REFERENCES category(category_id) ON DELETE SET NULL ON UPDATE CASCADE
);
""")

#receipt table
cursor.execute("""drop table if exists receipt """)
cursor.execute("""
CREATE TABLE IF NOT EXISTS receipt (
    receipt_id TEXT PRIMARY KEY,
    user_id TEXT,
    create_date DATE,
    scanned_date DATE,
    finished_date DATE,
    modify_date DATE,
    points_awarded_date DATE,
    purchase_date DATE,
    bonus_points_earned INTEGER,
    points_earned INTEGER,
    purchase_item_count INTEGER,
    total_spent INTEGER,
    bonus_reason_id INTEGER,
    receipt_status_id INTEGER,
    FOREIGN KEY (user_id) REFERENCES user(user_id) ON DELETE SET NULL ON UPDATE CASCADE,
    FOREIGN KEY (bonus_reason_id) REFERENCES bonus_reason(bonus_reason_id) ON DELETE SET NULL ON UPDATE CASCADE,
    FOREIGN KEY (receipt_status_id) REFERENCES receipt_status(receipt_status_id) ON DELETE SET NULL ON UPDATE CASCADE
);
""")

#item table
cursor.execute("""
CREATE TABLE IF NOT EXISTS item (
    barcode TEXT PRIMARY KEY,
    brand_id TEXT,
    brand_code TEXT,
    description TEXT,
    final_price REAL,
    quantity_purchased INTEGER,
    FOREIGN KEY (brand_id) REFERENCES brand(brand_id) ON DELETE SET NULL ON UPDATE CASCADE
);
""")

#item receipt table (bridge table to connect receipt and item)
cursor.execute("""drop table if exists receipt_item """)
cursor.execute("""
CREATE TABLE IF NOT EXISTS receipt_item (
    receipt_id INTEGER,
    barcode TEXT,
    PRIMARY KEY (receipt_id, barcode),
    FOREIGN KEY (receipt_id) REFERENCES receipt(receipt_id) ON DELETE SET NULL ON UPDATE CASCADE,
    FOREIGN KEY (barcode) REFERENCES item(barcode) ON DELETE SET NULL ON UPDATE CASCADE
);
""")


for bonus_reason in set(receipts_df['bonusPointsEarnedReason']):
    cursor.execute('INSERT OR IGNORE INTO bonus_reason (bonus_reason_name) VALUES (?)', (bonus_reason,))
for receipt_status in set(receipts_df['rewardsReceiptStatus']):
    cursor.execute('INSERT OR IGNORE INTO receipt_status (receipt_status_name) VALUES (?)', (receipt_status,))
for state in set(users_df[users_df['state'].notna()]['state']):
    cursor.execute('INSERT OR IGNORE INTO state (state_name) VALUES (?)', (state,))
for role in set(users_df['role']):
    cursor.execute('INSERT OR IGNORE INTO role (role_name) VALUES (?)', (role,))
for sign_up_source in set(users_df[users_df['signUpSource'].notna()]['signUpSource']):
    cursor.execute('INSERT OR IGNORE INTO sign_up_source (sign_up_source_name) VALUES (?)', (sign_up_source,))
for category, category_code in brands_category_list:
    cursor.execute('INSERT OR IGNORE INTO category (category_name, category_code) VALUES (?,?)', (category, category_code))

for index, row in users_df.iterrows():
    user_id = row['_id']
    active = row['active']
    created_date = row['createdDate']
    last_login_date = row['lastLogin']
    role_id = get_id('role', row['role'], 'role_name')
    if pd.isna(row['state']):
        state_id = np.nan
    else:
        state_id = get_id('state', row['state'], 'state_name')
    if pd.isna(row['signUpSource']):
        sign_up_source_id = np.nan
    else:
        sign_up_source_id = get_id('sign_up_source', row['signUpSource'], 'sign_up_source_name')
    cursor.execute('INSERT OR REPLACE INTO user (user_id, active, created_date, last_login_date, role_id, state_id, sign_up_source_id) VALUES (?,?,?,?,?,?,?)', (user_id, active, created_date, last_login_date, role_id, state_id, sign_up_source_id))
        
for index, row in brands_df.iterrows():
    brand_id = row['_id']
    barcode = row['barcode']
    cpg = row['cpg']
    name = row['name']
    top_brand = row['topBrand']
    if pd.isna(row['brandCode']):
        brand_code = np.nan
    else:
        brand_code = row['brandCode']
    category_id = get_id('category', row['category'],'category_name', row['categoryCode'], 'category_code')
    cursor.execute('INSERT OR REPLACE INTO brand (brand_id, barcode, cpg, name, top_brand, brand_code, category_id) VALUES (?,?,?,?,?,?,?)', (brand_id, barcode, cpg, name, top_brand, brand_code, category_id))


for index, row in receipts_df.iterrows():
    receipt_id = row['_id']
    create_date = row['createDate']
    scanned_date = row['dateScanned']
    finished_date = row['finishedDate']
    modify_date =row['modifyDate']
    points_awarded_date =row['pointsAwardedDate']
    purchase_date =row['purchaseDate']
    bonus_points_earned =row['bonusPointsEarned']
    points_earned =row['pointsEarned']
    purchase_item_count =row['purchasedItemCount']
    total_spent =row['totalSpent']
    user_id = get_id('user',row['userId'],'user_id')
    bonus_reason_id = get_id('bonus_reason', row['bonusPointsEarnedReason'], 'bonus_reason_name')
    receipt_status_id =get_id('receipt_status', row['rewardsReceiptStatus'], 'receipt_status_name')
    cursor.execute('INSERT OR REPLACE INTO receipt (receipt_id, create_date, scanned_date, finished_date, modify_date, points_awarded_date, purchase_date,bonus_points_earned, points_earned, purchase_item_count,total_spent,bonus_reason_id, receipt_status_id,user_id ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)', (receipt_id, create_date, scanned_date, finished_date, modify_date, points_awarded_date, purchase_date,bonus_points_earned, points_earned, purchase_item_count,total_spent,bonus_reason_id, receipt_status_id, user_id ))

for index, row in receipts_df.iterrows():
    if pd.isna(row['rewardsReceiptItemList']):
        continue
    for item in ast.literal_eval(row['rewardsReceiptItemList']):
        if 'barcode' in item:
            barcode = item['barcode']
        if 'brandCode' in item:
            brand_code = item['brandCode']
        else:
            brand_code = np.nan
        if 'description' in item:
            description = item['description']
        else:
            description = np.nan
        #try search from description to see if we can match brandcode by using name from brands_df vs. description from item
        if pd.isna(brand_code) and not pd.isna(description) and description.lower() != 'item not found':
#             print(description)
            brand_code = get_brandcode(description)
#             print(brand_code)

        if 'quantityPurchased' in item:
            quantity_purchased = item['quantityPurchased']
        else:
            quantity_purchased = np.nan
        if 'finalPrice' in item:
            final_price = item['finalPrice']
        else:
            final_price = np.nan
        if not pd.isna(brand_code):
            
            brand_id = get_brand_id(brand_code)
        else:
            brand_id = np.nan
        cursor.execute('INSERT OR IGNORE INTO item (barcode, brand_id, brand_code, description, final_price, quantity_purchased) VALUES (?,?,?,?,?,?)', (barcode,brand_id, brand_code, description, final_price, quantity_purchased))
    

for index, row in receipts_df.iterrows():
    receipt_id = row['_id']
    if pd.isna(row['rewardsReceiptItemList']):
        continue
    else:
        for item in ast.literal_eval(row['rewardsReceiptItemList']):
            if 'barcode' in item:
                barcode = item['barcode']
                cursor.execute('INSERT OR IGNORE INTO receipt_item (receipt_id, barcode) VALUES (?,?)', (receipt_id, barcode))

conn.commit()

conn.close()



### Save all relational tables to a folder

In [10]:
db_path = 'receipts.db'
output_folder = 'receipt_schema_tables'
conn = sqlite3.connect(db_path)

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

conn = sqlite3.connect(db_path)

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table in tables:
    table_name = table[0] 
    query = f"SELECT * FROM {table_name};" 
    
    try:
        df = pd.read_sql_query(query, conn)
        
        csv_file = os.path.join(output_folder, f"{table_name}.csv")
        
        df.to_csv(csv_file, index=False)
        print(f"Exported {table_name} to {csv_file}")
    except Exception as e:
        print(f"Error exporting {table_name}: {e}")

conn.close()

Exported sqlite_sequence to receipt_schema_tables/sqlite_sequence.csv
Exported sign_up_source to receipt_schema_tables/sign_up_source.csv
Exported bonus_reason to receipt_schema_tables/bonus_reason.csv
Exported receipt_status to receipt_schema_tables/receipt_status.csv
Exported state to receipt_schema_tables/state.csv
Exported role to receipt_schema_tables/role.csv
Exported user to receipt_schema_tables/user.csv
Exported category to receipt_schema_tables/category.csv
Exported brand to receipt_schema_tables/brand.csv
Exported receipt to receipt_schema_tables/receipt.csv
Exported item to receipt_schema_tables/item.csv
Exported receipt_item to receipt_schema_tables/receipt_item.csv
